In [ ]:
import utils
import models.categorical
import math
import pickle
import numpy as np
import pandas as pd
from datetime import date

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

# Hyperparameter Tuning

In [ ]:
futuresList = utils.futuresList

logregParams = [
    {
        "C": [0.1, 0.5, 1.0, 1.5],
        "solver": ["liblinear"],
        "penalty": ["l1", "l2"],
        "class_weight": ["balanced", None] 
    },
    {
        "C": [0.1, 0.5, 1.0, 1.5],
        "solver": ["lbfgs", "newton-cg"],
        "penalty": ["l2", "none"],
        "class_weight": ["balanced", None] 
    }
]

start_date = date(2010, 1, 1)
end_date = date(2021, 1, 1)

In [ ]:
# retrieve parameter grid
parameter_grid = list(ParameterGrid(logregParams))
X_vars = ["CLOSE_PCT", "VOL_PCT"]
y_var = "LONG_SHORT"
file_dir = "logreg/perc/"

for future in tqdm(futuresList):
    print(future)
    # load data
    df = utils.prepare_data(future)
    
    # load X and y
    X_df, y_df = utils.generate_X_y(df, X_vars=X_vars, y_var=y_var)
    cost_df = df["CLOSE_PCT"]
    
    # prepare collated results
    agg_results_collated = pd.DataFrame(index=list(range(len(parameter_grid))), 
                                        columns=["accuracy_SMA", "opp_cost_SMA"])
    win_results_collated = []

    # run walk forward validation 
    for i in range(len(parameter_grid)):
        param_set = parameter_grid[i]
        model = LogisticRegression(**param_set)
        win_results, agg_results = models.categorical.walk_forward(
            model = model, X = X_df, y = y_df, cost_weight = cost_df, rolling = True, 
            max_windows = 100, start_index = start_date
        )
        win_results_collated.append(win_results)
        agg_results_collated.loc[i, "accuracy_SMA"] = agg_results.loc["SMA", "accuracy"]
        agg_results_collated.loc[i, "opp_cost_SMA"] = agg_results.loc["SMA", "opp_cost"]

    # save parameters
    parameter_df = pd.DataFrame.from_records(parameter_grid)
    combined_df = pd.concat([parameter_df, agg_results_collated], axis=1)
    combined_df = combined_df.sort_values(by=["accuracy_SMA", "opp_cost_SMA"], ascending=False)
    combined_df.to_csv(f"model_metrics/categorical/{file_dir}{future}.csv", index=False)

# Save Models

In [ ]:
import utils
import models.categorical
import pickle
import numpy as np
import pandas as pd
from datetime import date

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import ParameterGrid
from tqdm import tqdm

import warnings
warnings.filterwarnings("ignore")

In [ ]:
# instantiate variables
futuresList = utils.futuresList

for future in futuresList:
    models.categorical.save_model(path='logreg/perc', metric="opp_cost_SMA", 
                                  model_fn=LogisticRegression, model_wrapper=models.categorical.LogRegWrapper, 
                                  future=future, X_vars=["CLOSE_PCT", "VOL_PCT"], y_var="LONG_SHORT")